# Решение кейса "Рекомендательная система видео" 
## Кейсодержатель: RUTUBE

In [180]:
import pandas as pd
import os
import gc

print("loading data ...")
hack = pd.read_csv("small_player_starts_train.csv")

loading data ...


In [181]:
hack.head(5)

,date,user_id,item_id,watch_time,is_autorized
0,2023-08-21 15:53:30+03:00,user_7941459,video_1535803,22,0
1,2023-08-21 16:17:58+03:00,user_17893165,video_796847,374,0
2,2023-08-21 21:18:12+03:00,user_25219604,video_1905049,11,0
3,2023-08-21 01:06:10+03:00,user_24477183,video_1156618,1,0
4,2023-08-21 17:24:32+03:00,user_25483180,video_500499,20,0


#### Для решения возьмем данные о просмотрах пользователей и метаданные о видео

In [182]:
print("Amount of events: ", len(hack))
users = hack.user_id.unique()
print("Unique users: ", len(users))
print("Unique items: ", hack.item_id.nunique())
print("Min date: ", hack.date.min())
print("Max date: ", hack.date.max())

Amount of events:  2439644
Unique users:  1313891
Unique items:  261288
Min date:  2023-08-21 00:00:00+03:00
Max date:  2023-08-21 23:59:59+03:00


In [183]:
# preprocess date
hack['date_short'] = hack.date.apply(lambda l: l.split(" ")[0])

#### Отфильтруем только "активных" пользователей (тех, у кого более 1 просмотра за сессию, в нашем случае - за сутки).

In [184]:
%%time
groupby = hack.groupby(["user_id", "date_short"]).count()
users_active = groupby[groupby['item_id'] >= 2].index.unique()

CPU times: user 2.54 s, sys: 134 ms, total: 2.67 s
Wall time: 2.76 s


In [185]:
users_active = set(users_active)

In [186]:
from tqdm import tqdm 
tqdm.pandas()
hack['active'] = hack.progress_apply(lambda l: True if (l.user_id, l.date_short) in users_active else False, axis=1)
hack=hack[hack.active]

100%|██████████████████████████████| 2439644/2439644 [00:25<00:00, 95408.01it/s]


## Необходимые датасеты

In [385]:
hack.reset_index(drop= True , inplace= True )

In [188]:
hack

,date,user_id,item_id,watch_time,is_autorized,date_short,active
0,2023-08-21 15:53:30+03:00,user_7941459,video_1535803,22,0,2023-08-21,True
1,2023-08-21 21:18:12+03:00,user_25219604,video_1905049,11,0,2023-08-21,True
2,2023-08-21 17:24:32+03:00,user_25483180,video_500499,20,0,2023-08-21,True
3,2023-08-21 15:21:19+03:00,user_22148320,video_1366097,32,0,2023-08-21,True
4,2023-08-21 00:35:41+03:00,user_14452025,video_283933,2142,0,2023-08-21,True
...,...,...,...,...,...,...,...
1457171,2023-08-21 16:44:47+03:00,user_11895299,video_853811,101,0,2023-08-21,True
1457172,2023-08-21 02:51:53+03:00,user_15478739,video_1449287,291,1,2023-08-21,True
1457173,2023-08-21 05:19:55+03:00,user_3507470,video_464555,261,0,2023-08-21,True
1457174,2023-08-21 12:32:32+03:00,user_13128840,video_420973,21,0,2023-08-21,True


In [9]:
features = pd.read_parquet("videos.parquet")

In [179]:
features

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
0,video_165654,MSI Pro MP241X недоОБЗОР (РЕШЕНИЕ ПРОБЛЕМЫ С М...,Silvi,None,0,В видео я обывательским взглядом расскажу про ...,Технологии и интернет,2022-12-08 13:53:05+03:00,391382,0,0,NaN,0.0,0.000000,NaN
1,video_1173704,Наложение пястно фаланговой повязки на кисть,"УЦ ""Академия Безопасности""",None,0,Видео с канала УЦ Академия безопасности (ab-dp...,Образование,2022-03-24 09:19:15+03:00,125922,26,0,NaN,0.0,0.000000,0.00
2,video_23927,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Artur48,None,0,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Авто-мото,2022-03-19 17:41:49+03:00,436570,2,0,NaN,NaN,0.000000,0.00
3,video_1003780,БОЛЬНИЦА в Brookhaven! ДОКТОР ПУПКИН спас ЖЕНИ...,ПАПА ДОЧКИ Games,None,0,Играем в Роблокс (Roblox) - БОЛЬНИЦА в Brookha...,Детям,2021-02-20 11:50:53+03:00,719377,673,0,0.0,1.0,0.000000,0.00
4,video_105383,"Вебинар ""Особенности трудоустройства граждан Б...","ЗАО ""Сплайн-Центр""",None,0,"10.08.2023 Вебинар ""Особенности трудоустройств...",Бизнес и предпринимательство,2023-08-11 09:02:07+03:00,3834404,19,0,0.0,0.0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320655,video_839160,Любимцы: Гена в памперсе бегает по клинике,Любимцы,Любимцы,1,None,Сериалы,2017-09-11 22:00:13+03:00,23290,0,0,NaN,0.0,0.333333,0.25
2320656,video_2134427,"Телеканал «СПАС», «Главное», интервью С. В. Ст...",Фонд развития территорий,None,0,None,Интервью,2022-03-28 11:19:33+03:00,1548792,57,0,0.0,0.0,0.000000,0.00
2320657,video_1453482,Взятие 3 отметок на ИС 7,CeperA_TyLyH,None,0,None,Видеоигры,2022-09-10 20:05:17+03:00,520001,12,0,NaN,NaN,NaN,NaN
2320658,video_2225207,Простой и вкусный рецепт пирога с грушами и сы...,Комедийные Шоу и Скетчи ❤,None,0,None,Разное,2023-07-06 12:16:29+03:00,8267,9,0,0.0,0.0,0.000000,0.00


### Подготовка данных

In [382]:
import numpy as np

item = np.array(features['item_id'].to_list())
item_h = np.array(hack['item_id'].to_list())
#categ = np.array(features['category_title'].to_list())
authors = np.array(features['author_title'].to_list())
hack1 = hack
hack1

In [520]:
#%%time
#аhack1['cat'] = [categ[np.where(item == i)][0] for i in tqdm(item_h)]

In [384]:
hack1['author'] = [authors[np.where(item == i)][0] for i in tqdm(item_h)]

100%|███████████████████████████████| 1457176/1457176 [5:14:44<00:00, 77.16it/s]


In [387]:
hack1.to_csv('hack.csv')

## Модель
### Построим модель рекомендаций, основанную на совместной встречаемости видео, категорий видео и определенных авторов в сессиях пользователей

В качеcтве сессии будем брать просмотры пользователя за сутки

In [358]:
def calc_pairs_video(train):
    print("Get sessions")
    count = train.groupby(['item_id']).size().rename('count_source').reset_index()[
        ['item_id', 'count_source']]
    dt = train.groupby(['user_id', 'date_short'])['item_id'].agg(list).rename('pair').reset_index()
    df = train[['user_id', 'date_short', 'item_id']].merge(dt, on=['user_id', 'date_short'], how='left')
        
    
    del dt
    gc.collect()

    # Explode the rows vs list of articles
    print("Explode the rows vs list of articles")
    df = df[['item_id', 'pair']].explode(column='pair')
    gc.collect()
    
    print("Discard duplicates")
    df = df.loc[df['item_id'] != df['pair']].reset_index(drop=True)
    
    print("Count how many times each pair combination happens")
    df = df.groupby(['item_id', 'pair']).size().rename('count').reset_index()
    
    print("join with frecuency of source item")
    df = df.merge(count, on=['item_id'], how='left')
    count = count.rename({'count_source': 'count_target', 'item_id': 'pair'}, axis=1)
    df = df.merge(count, on=['pair'], how='left')

    df['score'] = df.apply(lambda l: l['count'] / (l['count_source']*l['count_target']), axis=1)
    df.sort_values(by=['score'], ascending=False, inplace=True)
    gc.collect()
    return df

In [359]:
def calc_pairs_category(train):
    print("Get sessions")
    count = train.groupby(['cat']).size().rename('count_source').reset_index()[
        ['cat', 'count_source']]
    dt = train.groupby(['user_id', 'date_short'])['cat'].agg(list).rename('pair').reset_index()
    df = train[['user_id', 'date_short', 'cat']].merge(dt, on=['user_id', 'date_short'], how='left')
        
    
    del dt
    gc.collect()

    # Explode the rows vs list of articles
    print("Explode the rows vs list of articles")
    df = df[['cat', 'pair']].explode(column='pair')
    gc.collect()
    
    print("Discard duplicates")
    df = df.loc[df['cat'] != df['pair']].reset_index(drop=True)
    
    print("Count how many times each pair combination happens")
    df = df.groupby(['cat', 'pair']).size().rename('count').reset_index()
    
    print("join with frecuency of source item")
    df = df.merge(count, on=['cat'], how='left')
    count = count.rename({'count_source': 'count_target', 'cat': 'pair'}, axis=1)
    df = df.merge(count, on=['pair'], how='left')

    df['score'] = df.apply(lambda l: l['count'] / (l['count_source']*l['count_target']), axis=1)
    df.sort_values(by=['score'], ascending=False, inplace=True)
    gc.collect()
    return df

In [389]:
def calc_pairs_authors(train):
    print("Get sessions")
    count = train.groupby(['author']).size().rename('count_source').reset_index()[
        ['author', 'count_source']]
    dt = train.groupby(['user_id', 'date_short'])['author'].agg(list).rename('pair').reset_index()
    df = train[['user_id', 'date_short', 'author']].merge(dt, on=['user_id', 'date_short'], how='left')
        
    
    del dt
    gc.collect()

    # Explode the rows vs list of articles
    print("Explode the rows vs list of articles")
    df = df[['author', 'pair']].explode(column='pair')
    gc.collect()
    
    print("Discard duplicates")
    df = df.loc[df['author'] != df['pair']].reset_index(drop=True)
    
    print("Count how many times each pair combination happens")
    df = df.groupby(['author', 'pair']).size().rename('count').reset_index()
    
    print("join with frecuency of source item")
    df = df.merge(count, on=['author'], how='left')
    count = count.rename({'count_source': 'count_target', 'author': 'pair'}, axis=1)
    df = df.merge(count, on=['pair'], how='left')

    df['score'] = df.apply(lambda l: l['count'] / (l['count_source']*l['count_target']), axis=1)
    df.sort_values(by=['score'], ascending=False, inplace=True)
    gc.collect()
    return df

In [441]:
%%time
video = calc_pairs_video(hack1[['user_id', 'item_id', 'date_short']])

Get sessions
Explode the rows vs list of articles
Discard duplicates
Count how many times each pair combination happens
join with frecuency of source item
CPU times: user 2min 2s, sys: 24.5 s, total: 2min 27s
Wall time: 3min 7s


In [442]:
video

,item_id,pair,count,count_source,count_target,score
0,video_10000,video_1551431,1,1,1,1.000000e+00
4882252,video_1976369,video_2066514,1,1,1,1.000000e+00
4882212,video_1976369,video_1821217,1,1,1,1.000000e+00
4882214,video_1976369,video_1826436,1,1,1,1.000000e+00
4882215,video_1976369,video_1834620,1,1,1,1.000000e+00
...,...,...,...,...,...,...
9123663,video_68646,video_1508131,1,20629,5298,9.149764e-09
9125703,video_68646,video_803844,1,20629,5385,9.001940e-09
9695086,video_803844,video_68646,1,5385,20629,9.001940e-09
2539169,video_1508623,video_1761620,1,17433,9386,6.111493e-09


In [362]:
#%%time
#categories = calc_pairs_category(hack1[['user_id', 'cat', 'date_short']])

Get sessions
Explode the rows vs list of articles
Discard duplicates
Count how many times each pair combination happens
join with frecuency of source item
CPU times: user 398 ms, sys: 5.95 ms, total: 404 ms
Wall time: 404 ms


In [ ]:
#categories

In [392]:
%%time
authors = calc_pairs_authors(hack1[['user_id', 'author', 'date_short']])

Get sessions
Explode the rows vs list of articles
Discard duplicates
Count how many times each pair combination happens
join with frecuency of source item
CPU times: user 56.4 s, sys: 12.6 s, total: 1min 8s
Wall time: 1min 14s


In [400]:
authors

,author,pair,count,count_source,count_target,score
913893,Астрологи России,Хочу всё знать!,1,1,1,1.000000e+00
1099029,Дневник деревенской домохозяйки,channel32125286,1,1,1,1.000000e+00
1099015,Дневник деревенской домохозяйки,artursafin,1,1,1,1.000000e+00
1099017,Дневник деревенской домохозяйки,astrolobster,1,1,1,1.000000e+00
165723,ElenaSh,Kleopatra,2,1,2,1.000000e+00
...,...,...,...,...,...,...
1799476,Тыгыдык!,Телеканал ТНТ,5,8155,289943,2.114625e-09
1784343,Телеканал ТНТ,Катерина Измайлова,1,289943,2423,1.423423e-09
1240537,Катерина Измайлова,Телеканал ТНТ,1,2423,289943,1.423423e-09
1044771,Гедонист,Телеканал ТНТ,1,8327,289943,4.141892e-10


# Submission

### Cделаем предсказания для тестовых пользователей, будем притягивать рекомендации к последнему видео в трейне следующим образом: к последнему видео рекомендуется ряды видео категорий и авторов, часто встречаемых с автором и категорией данного ролика

Для пользователей, которые не попали в трейн, но есть на тесте, порекомендуем самые популярные видео из трейна

#### Функции для поиска последнего видео категорий и авторов 

In [421]:
def auth_pop(author, features):
    auth = features.loc[features['author_title'] == author][['item_id', 'publicated']].sort_values(by=['publicated'], ascending = [False])
    return auth['item_id'].tolist()[0]
    
    
#def cat_pop(cat, features):
    #cats = features.loc[features['category_title'] == cat]['item_id'].sort_values(by = ['publicated'], ascending = [False])
    #return cats['item_id'].tolist()[:num]

#### Самые популярные видео для холодных пользователей

In [499]:
most_popular = list(hack1.item_id.value_counts()[:10].to_frame().index)

In [403]:
features[features.item_id.isin(most_popular)][:5]

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
71943,video_2171643,Денис Власенко о школьном буллинге,Вокруг•ТВ,None,0,Денис Власенко дал эксклюзивное интервью «Вокр...,Лайфстайл,2023-08-17 14:36:37+03:00,61280,2146,0,NaN,NaN,NaN,0.000000
155423,video_68646,"Выжить в Дубае, 8 выпуск",Телеканал ТНТ,Выжить в Дубае,1,Премьера! «Выжить в Дубае» – новое масштабное ...,Телепередачи,2023-08-13 21:00:12+03:00,5208840,75496,0,NaN,NaN,NaN,0.411112
162112,video_170129,Филипп Янковский высказался об Андрее Тарковском,Вокруг•ТВ,None,0,Филипп Янковский с детства впитывал в себя маг...,Искусство,2023-07-14 18:54:34+03:00,145898,2146,0,0.000000,0.000000,0.000000,0.000000
307013,video_1508131,Анастасия Попова («Бэби-тур») высказалась об о...,Вокруг•ТВ,None,0,В 2023 году актриса снялась в комедии положени...,Лайфстайл,2023-07-21 18:27:19+03:00,99883,2146,0,0.078947,0.010526,0.010309,0.000000
406823,video_1594159,"Выжить в Дубае, 7 выпуск",Телеканал ТНТ,Выжить в Дубае,1,Премьера! «Выжить в Дубае» – новое масштабное ...,Телепередачи,2023-08-06 21:00:18+03:00,5930640,75496,0,NaN,NaN,0.448765,0.319222


#### Находим последнее просмотренное видео пользователями из теста (которые также есть и в трейне)

In [404]:
hack1 = hack1[hack1.user_id.isin(hack1['user_id'].unique())].sort_values(by=['date'])

In [431]:
%%time
last_watched_videos = (
    hack[['user_id', 'item_id']]
    .groupby(['user_id'], as_index=False)
    .agg('last')
)

CPU times: user 696 ms, sys: 118 ms, total: 813 ms
Wall time: 911 ms


In [432]:
last_watched_videos = last_watched_videos.set_index('user_id').to_dict()['item_id']

In [445]:
items_in_train = {source: list(group.index) for source, group in video.groupby(['item_id'])}
#cats_in_train = {source: list(group.index) for source, group in categories.groupby(['cat'])}
authors_in_train = {source: list(group.index) for source, group in authors.groupby(['author'])}

### Решение

In [480]:
sub = pd.read_csv('sample_submission.csv')
sub = sub.iloc[:10000, :]

In [481]:
from tqdm import tqdm 
tqdm.pandas()


def get_recs_for_users(video, authors,  sample_submission, last_watched_videos, items_in_train, num_candidates: int=5):
    
    for i, row in tqdm(sample_submission.iterrows()):
        if row.user_id in last_watched_videos:
            videoo = last_watched_videos[row.user_id]
            video_ids = items_in_train[(videoo,)] if (videoo,) in items_in_train else ''
            aut_ids = authors_in_train[(features.loc[features['item_id'] == videoo]['author_title'].values[0],)] if (features.loc[features['item_id'] == videoo]['author_title'].values[0],) in authors_in_train else ''
            #cat_ids = cats_in_train[(features.loc[features['item_id'] == video]['category_title'].values[0],)] if (features.loc[features['item_id'] == video]['category_title'].values[0],) in cats_in_train else ''
            if len(video_ids)>0 and len(aut_ids)>0:
                recs = []
                recs.extend(video.iloc[video_ids].item_id.to_list()[:num_candidates])
                auth = authors.iloc[aut_ids].author.to_list()[:num_candidates]
                #catts = categories.iloc[cat_ids].item_id.to_list()[:num_candidates]
                for i in range(num_candidates):
                    if i < len(auth):
                        recs.extend([auth_pop(auth[i], features)])
                sample_submission.loc[i, 'recs'] = recs
            else:
                sample_submission.loc[i, 'recs'] = most_popular
        else:
            sample_submission.loc[i, 'recs'] = most_popular
    return sample_submission

In [482]:
submission = get_recs_for_users(video, authors, sub, last_watched_videos, items_in_train)

10000it [50:20,  3.31it/s]


In [512]:
submission

,user_id,recs
0,user_26511551,"['video_0', 'video_0', 'video_0', 'video_0', '..."
1,user_29194819,"[video_283933, video_68646, video_1508623, vid..."
2,user_29734049,"[video_283933, video_68646, video_1508623, vid..."
3,user_955460,"[video_283933, video_68646, video_1508623, vid..."
4,user_7065521,"[video_289452, video_2305793, video_1198109, v..."
...,...,...
9995,user_7999467,"['video_0', 'video_0', 'video_0', 'video_0', '..."
9996,user_29463140,"[video_283933, video_68646, video_1508623, vid..."
9997,user_21225638,"['video_0', 'video_0', 'video_0', 'video_0', '..."
9998,user_10637596,"[video_283933, video_68646, video_1508623, vid..."


In [521]:
sub = pd.read_csv('sample_submission.csv')
sub = sub.iloc[10000:, :]
sub.loc[:, 'recs'] = "['" + "', '".join(most_popular) + "']"

In [522]:
sub_itog =pd.concat([submission, sub.iloc[:, :]],ignore_index=True)

In [523]:
sub_itog

,user_id,recs
0,user_26511551,"['video_0', 'video_0', 'video_0', 'video_0', '..."
1,user_29194819,"[video_283933, video_68646, video_1508623, vid..."
2,user_29734049,"[video_283933, video_68646, video_1508623, vid..."
3,user_955460,"[video_283933, video_68646, video_1508623, vid..."
4,user_7065521,"[video_289452, video_2305793, video_1198109, v..."
...,...,...
97235,user_29281681,"['video_283933', 'video_68646', 'video_1508623..."
97236,user_3912848,"['video_283933', 'video_68646', 'video_1508623..."
97237,user_28389099,"['video_283933', 'video_68646', 'video_1508623..."
97238,user_18951296,"['video_283933', 'video_68646', 'video_1508623..."


In [524]:
sub_itog.to_csv("submission.csv", index=False)